<a href="https://colab.research.google.com/github/fezilemahlangu/Reinforcement-Learning-Project/blob/master/MCTS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Installing Dependencies

In [ ]:
#https://towardsdatascience.com/deep-reinforcement-learning-and-monte-carlo-tree-search-with-connect-4-ba22a4713e7a

#https://ai-boson.github.io/mcts/


In [1]:
!apt update
!apt install -y cmake
!apt-get install -y build-essential autoconf libtool pkg-config
!apt-get install flex bison libbz2-dev
!pip install nle
!pip install minihack
!python -m minihack.scripts.env_list
!pip install gym[atari,accept-rom-license]

Get:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:5 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:7 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:8 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:9 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [83.3 kB]
Get:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease [15.9 kB]
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:14 http://security.ubuntu.c

# Libraries

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import gym
import nle
import minihack
from gym import spaces
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam
import torch
import random
from minihack import RewardManager
from collections import defaultdict
import warnings
warnings.filterwarnings("ignore")

# MCTS

In [21]:
#reward shaping 

reward_manager = RewardManager()
reward_manager.add_eat_event("apple", reward=1)
reward_manager.add_wield_event("dagger", reward=2)
reward_manager.add_wield_event("wand", reward=20)
reward_manager.add_kill_event("minotaur",reward=40)
reward_manager.add_location_event("sink", reward=-1, terminal_required=False)

env = gym.make("MiniHack-Quest-Hard-v0",reward_manager=reward_manager)

# Node

In [20]:
class State():

  def __init__(self, state, reward=0,done=False,info=None):
        # print('init state')
        self.state_s = state
        self.reward = reward
        self.done = done
        self.info = info
        if done:
            self.legal_actions = []
        else:
            self.legal_actions = list(np.arange(env.action_space.n))

class MCTS():
  '''
  Class for MCTS node 
  '''
  def __init__(self,state, parent=None,  parent_action=None):

    self.state = state
    self.parent = parent # parent node in MCTS
    self.parent_action = parent_action # action that the parent took
    self.children = [] #children of parent node 
    self.visit_count = 1 # keeps count of how many times node has been visited
    self.rewards = defaultdict(int)
    self.unexplored_actions = self.state.legal_actions

  def expand(self,env):

    '''
    expand node and take unexplored action 
    '''
    action = self.unexplored_actions.pop()

    obs, reward, done, _ = env.step(action)

    obs = State(obs,reward,done,_)

    child_node = MCTS(obs,self,action) #obs should be state 

    self.children.append(child_node)

    return child_node, env

  def back_propagate(self,reward):
    '''
    Back propagate reward on all nodes from leaf to root and update visit count 
    '''
    
    self.visit_count += 1
    self.rewards[reward] += 1

    if self.parent:
      self.parent.back_propagate(reward)


  def rollout_policy(self, moves):
    #randomly selects a child node 
    '''
    rollout policy 
    '''

    return moves[np.random.randint(len(moves))]

  
  def rollout(self,env):
    '''
    rollout: until we reach the leaf node, randomly choose an action at each step and simulate 
    this act to receive an ave reward until done (ep is over)
    '''
    curr = self.state

    while not curr.done:
      moves = curr.legal_actions

      act = self.rollout_policy(moves)

      obs, reward, done, _ = env.step(act)

      curr = State(obs,reward,done,_)
    
    return curr.reward

  def tree_policy(self,env): #->fix 
    '''
    keeps expanding tree until terminal node is reached 
    '''
    curr = self
    while not curr.state.done:
      if len(self.children) < env.action_space.n:
        return self.expand(env)
      else:
        curr = curr.best_child(c_p = 0.1)

    return curr, env
  
  def n(self):
    return self.visit_count

  def q(self):
    # wins = self._results[1]
    # loses = self._results[-1]
    # return wins - loses

    total = 0
    for r in self.rewards:
        total += r*self.rewards[r]

    return total

  def best_child(self, c_param=0.1):
    
    choices_weights = [(c.q() / c.n()) + c_param * np.sqrt((2 * np.log(self.n()) / c.n())) for c in self.children]
    return self.children[np.argmax(choices_weights)]

  def best_action(self,actions):
    simulation_no = 7
	
	
    for i in range(simulation_no):
      new_env = gym.make("MiniHack-Quest-Hard-v0",reward_manager=reward_manager)
      new_init_state = new_env.reset()

      for a in actions:
        obs,reward,done,_ = new_env.step(a)

      v = self.tree_policy(new_env)
      reward = v[0].rollout(v[1])
      v[0].back_propagate(reward)
	
    return self.best_child(c_param=0.1)

  
  # def best_child(self,c_p):
  #   '''
  #   finds best child using UCT
  #   xi is ave reward/value of all nodes beneath this node 
  #   ns is the numer of times the parent has been visited
  #   ni is the numer of times the child node i has been visited
  #   '''

  #   ns = self.visit_count #visit count of parent

  #   ni = [c.visit_count for c in self.children] #visit count of chilren 

  #   q = 0
  #   for r in self.rewards:
  #     q += r*self.rewards[r]

  #   first_term = q / ni

  #   second_term = c_p * np.sqrt((2*np.log(ns))/ni) 

  #   UCB1 = first_term + second_term 

  #   best_child = np.argmax(UCB1)

  #   return self.children[best_child]

  # def best_action(self):
  #   '''
  #   find next best action 
  #   '''

    
    



# Main

In [ ]:
def main():

    iters =1000

    for mpx in range(1):
        

        prev_actions = []
        rewards = []
        actionsTaken = []
        
        env = gym.make("MiniHack-Quest-Hard-v0",reward_manager=reward_manager)
        rewardArr = []
        
        initial_state = env.reset()
        initial_state = State(state = initial_state)
        # env.render()

        root = MCTS(state = initial_state)
     
        selected_node = root.best_action(prev_actions)
       
        prev_actions.append(selected_node.parent_action)


        next_state, reward, done, info = env.step(selected_node.parent_action)

        # rewards.append(reward)
        next_state = State(next_state,reward,done,info)

    
        cnt = 0
        
        # loops through the rest of the game repeatedly calling the 
        # best_action function and making the selected move in the environment
        for it in range(iters):
            if done:
                break;
            cnt += 1
            print("Current Game {0}, current step {1}".format(mpx, cnt))
            selected_node = MCTS(state = next_state)
            selected_node = selected_node.best_action(prev_actions)


            prev_actions.append(selected_node.parent_action)

            next_state, reward, done, info = env.step(selected_node.parent_action)
            rewards.append(reward)
            print("reward: {}".format(reward))

            next_state = State(next_state,reward,done,info)

        rewardArr = np.array(rewards)
        # saves the rewards received during game
        # np.savetxt("rewardArr-{0}".format(mpx), rewardArr)

if __name__ == "__main__":
  main()

Current Game 0, current step 1
reward: -0.01
Current Game 0, current step 2
reward: -0.01
Current Game 0, current step 3
reward: -0.01
Current Game 0, current step 4
reward: -0.01
Current Game 0, current step 5
reward: -0.01
Current Game 0, current step 6
reward: -0.01
Current Game 0, current step 7
reward: -0.01
Current Game 0, current step 8
reward: 0.0
Current Game 0, current step 9
reward: -0.01
Current Game 0, current step 10
reward: -0.01
Current Game 0, current step 11
reward: -0.01
Current Game 0, current step 12
reward: -0.01
Current Game 0, current step 13
reward: -0.01
Current Game 0, current step 14
reward: -0.01
Current Game 0, current step 15
reward: -0.01
Current Game 0, current step 16
reward: -0.01
Current Game 0, current step 17
reward: -0.01
Current Game 0, current step 18
reward: -0.01
Current Game 0, current step 19
reward: -0.01
Current Game 0, current step 20
reward: -0.01
Current Game 0, current step 21
reward: 0.0
Current Game 0, current step 22
reward: -0.01
C